## import usefull packages and functions

In [8]:
import json
import pandas as pd

from auchan_scraper.database import create_table, select_all
from auchan_scraper.get_categories import get_categories

### create database

In [2]:
create_table()

After creating tables we can start the search for all categories urls. For debugging purposes we use logging package. While scraping the urls of each subcategory, it is also saving the cookies for each, as without this process we would get response 400. After the process both categories and cookies are saved in json files (for future use in crawler)

In [3]:
url = "https://zakupy.auchan.pl/shop/artykuly-spozywcze.c-11908" # URL of the food categories page
categories, cookies = get_categories(url)

with open("auchan_scraper/categories.json", "w") as categories_file:
    json.dump(categories, categories_file)

with open("auchan_scraper/cookies.json", "w") as cookies_file:
    json.dump(cookies, cookies_file)

2024-04-04 17:36:30,193 [DEBUG] Selenium Manager binary found at: c:\Users\kaszt\OneDrive\Dokumenty\python nauka\SergioMaciek\auchan\lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe
2024-04-04 17:36:30,194 [DEBUG] Executing process: c:\Users\kaszt\OneDrive\Dokumenty\python nauka\SergioMaciek\auchan\lib\site-packages\selenium\webdriver\common\windows\selenium-manager.exe --browser chrome --debug --language-binding python --output json
2024-04-04 17:36:30,455 [DEBUG] chromedriver not found in PATH
2024-04-04 17:36:30,456 [DEBUG] chrome detected at C:\Program Files\Google\Chrome\Application\chrome.exe
2024-04-04 17:36:30,456 [DEBUG] Running command: wmic datafile where name='C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe' get Version /value
2024-04-04 17:36:30,457 [DEBUG] Output: "\r\r\n\r\r\nVersion=123.0.6312.88\r\r\n\r\r\n\r\r\n\r"
2024-04-04 17:36:30,457 [DEBUG] Detected browser: chrome 123.0.6312.88
2024-04-04 17:36:30,458 [DEBUG] Required driver: 

//*[@aria-label='Zamknij okno dialogowe'] Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF75B8E70C2+63090]
	(No symbol) [0x00007FF75B852D12]
	(No symbol) [0x00007FF75B6EEC65]
	(No symbol) [0x00007FF75B73499D]
	(No symbol) [0x00007FF75B734ADC]
	(No symbol) [0x00007FF75B775B37]
	(No symbol) [0x00007FF75B75701F]
	(No symbol) [0x00007FF75B773412]
	(No symbol) [0x00007FF75B756D83]
	(No symbol) [0x00007FF75B7283A8]
	(No symbol) [0x00007FF75B729441]
	GetHandleVerifier [0x00007FF75BCE262D+4238301]
	GetHandleVerifier [0x00007FF75BD1F78D+4488509]
	GetHandleVerifier [0x00007FF75BD17A6F+4456479]
	GetHandleVerifier [0x00007FF75B9C0606+953270]
	(No symbol) [0x00007FF75B85E5DF]
	(No symbol) [0x00007FF75B8592B4]
	(No symbol) [0x00007FF75B8593EB]
	(No symbol) [0x00007FF75B849C24]
	BaseThreadInitThunk [0x00007FFF3AC6257D+29]
	RtlUserThreadStart [0x00007FFF3BACAA58+40]



2024-04-04 17:36:46,073 [DEBUG] POST http://localhost:54462/session/9638cb332579615345e68ae3c7e857a8/element {'using': 'xpath', 'value': "//*[@id='accept-recommended-btn-handler']"}
2024-04-04 17:36:46,080 [DEBUG] http://localhost:54462 "POST /session/9638cb332579615345e68ae3c7e857a8/element HTTP/1.1" 200 0
2024-04-04 17:36:46,080 [DEBUG] Remote response: status=200 | data={"value":{"element-6066-11e4-a52e-4f735466cecf":"f.AD85532E371735109593B41E11F2525C.d.A65CA1E29F0632AAC7D45CE5C117721A.e.49"}} | headers=HTTPHeaderDict({'Content-Length': '126', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
2024-04-04 17:36:46,081 [DEBUG] Finished Request
2024-04-04 17:36:46,081 [DEBUG] POST http://localhost:54462/session/9638cb332579615345e68ae3c7e857a8/execute/sync {'script': '/* isDisplayed */return (function(){return (function(){var k=this||self;function aa(a){return"string...', 'args': [{'element-6066-11e4-a52e-4f735466cecf': 'f.AD85532E371735109593B41E11F2525C

//*[@id='accept-recommended-btn-handler'] Message: 



2024-04-04 17:36:56,124 [DEBUG] Remote response: status=200 | data={"value":"PRODUKTY ROŚLINNE"} | headers=HTTPHeaderDict({'Content-Length': '30', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
2024-04-04 17:36:56,124 [DEBUG] Finished Request
2024-04-04 17:36:56,125 [DEBUG] GET http://localhost:54462/session/9638cb332579615345e68ae3c7e857a8/element/f.AD85532E371735109593B41E11F2525C.d.A65CA1E29F0632AAC7D45CE5C117721A.e.61/text {}
2024-04-04 17:36:56,131 [DEBUG] http://localhost:54462 "GET /session/9638cb332579615345e68ae3c7e857a8/element/f.AD85532E371735109593B41E11F2525C.d.A65CA1E29F0632AAC7D45CE5C117721A.e.61/text HTTP/1.1" 200 0
2024-04-04 17:36:56,131 [DEBUG] Remote response: status=200 | data={"value":"NAPOJE"} | headers=HTTPHeaderDict({'Content-Length': '18', 'Content-Type': 'application/json; charset=utf-8', 'cache-control': 'no-cache'})
2024-04-04 17:36:56,132 [DEBUG] Finished Request
2024-04-04 17:36:56,133 [DEBUG] GET http://localhost:54462/s

## preview cookies

In [4]:
with open("auchan_scraper/cookies.json", "r") as cookies_file:
    cookies = json.load(cookies_file)
for cookie in cookies:
    for key in cookie:
        print(key, cookie[key], "\n")

domain .auchan.pl 

expiry 1746805071 

httpOnly False 

name _ga_12345 

path / 

sameSite Lax 

secure False 

value GS1.1.1712244993.1.1.1712245071.0.0.218968174 

domain .auchan.pl 

expiry 1712331471 

httpOnly False 

name _uetsid 

path / 

sameSite Lax 

secure False 

value 1d82bd20f29911ee956947643ea488d4 

domain .auchan.pl 

expiry 1746805066 

httpOnly True 

name FPID 

path / 

sameSite Lax 

secure True 

value FPID2.2.M%2B3uIIVjCAQB2kquJUKaDgVaIAZ%2BAehDWeFgDUNnUPM%3D.1712244993 

domain .auchan.pl 

expiry 1712331471 

httpOnly False 

name _clsk 

path / 

sameSite Lax 

secure False 

value dqi5vs%7C1712245071943%7C20%7C1%7Cf.clarity.ms%2Fcollect 

domain .auchan.pl 

expiry 1746805017 

httpOnly False 

name _ga 

path / 

sameSite Lax 

secure False 

value GA1.1.1492833240.1712244993 

domain .auchan.pl 

expiry 1720020990 

httpOnly False 

name FPAU 

path / 

sameSite Lax 

secure True 

value 1.1.1344542954.1712244995 

domain .auchan.pl 

expiry 1727796995 


## count number of subcategories

In [5]:
with open('auchan_scraper/categories.json', 'r') as f:
    categories = json.load(f)

url_count = 0
for category in categories:
    for subcategory in category['subcategories']:
        if 'url' in subcategory:
            url_count += 1
print(url_count)

157


## import spider to run without command line

In [6]:
from auchan_scraper.spiders.shop import ShopSpider
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings
import nest_asyncio

In ShopSpider we used following settings:
- ROBOTSTXT_OBEY = True (obey the robots.txt file: get response 200)
- DOWNLOAD_DELAY = 3 (3 second delay between each fetch of the data)
- COOKIES_ENABLED = True (without cookies user is not able to acces the api fetching data from the server)
- REQUEST_FINGERPRINTER_IMPLEMENTATION = "2.7"
- TWISTED_REACTOR = "twisted.internet.asyncioreactor.AsyncioSelectorReactor"
- FEED_EXPORT_ENCODING = "utf-8"

We also used additional middlewares to rotate headers:
- 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware' : None,
- 'scrapy_user_agents.middlewares.RandomUserAgentMiddleware' : 400

And custom pipelines:
- "auchan_scraper.pipelines.DuplicatesPipeline": 100, (removes duplicated items)
- "auchan_scraper.pipelines.SavingTosqlitePipeline": 200 (saves yield data to SQLite db)

To run scrapy in jupyter notebook in order to encounter error: "RuntimeError: This event loop is already running" we can use asyncio to create a event loop that can coexist with the one being used by Jupyter

In [7]:
nest_asyncio.apply()
process = CrawlerProcess(get_project_settings())

process.crawl(ShopSpider, number=3)

process.start()


2024-04-04 17:38:16,031 [INFO] Scrapy 2.11.1 started (bot: auchan_scraper)
2024-04-04 17:38:16 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: auchan_scraper)
2024-04-04 17:38:16,032 [INFO] Versions: lxml 5.1.0.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 23.10.0, Python 3.10.7 (tags/v3.10.7:6cc6b13, Sep  5 2022, 14:08:36) [MSC v.1933 64 bit (AMD64)], pyOpenSSL 24.0.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 42.0.5, Platform Windows-10-10.0.22631-SP0
2024-04-04 17:38:16 [scrapy.utils.log] INFO: Versions: lxml 5.1.0.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 23.10.0, Python 3.10.7 (tags/v3.10.7:6cc6b13, Sep  5 2022, 14:08:36) [MSC v.1933 64 bit (AMD64)], pyOpenSSL 24.0.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 42.0.5, Platform Windows-10-10.0.22631-SP0
2024-04-04 17:38:16,034 [INFO] Enabled addons:
[]
2024-04-04 17:38:16 [scrapy.addons] INFO: Enabled addons:
[]
2024-04-04 17:38:16,036 [DEBUG] Using reactor: twisted.internet.a

Connected to the database
13612 https://zakupy.auchan.pl/shop/artykuly-spozywcze/owoce-warzywa-ziola/ziola-i-kielki.c-13612
27394 https://zakupy.auchan.pl/shop/artykuly-spozywcze/kuchnie-swiata/produkty-kuchni-hiszpanskiej.c-27394
27382 https://zakupy.auchan.pl/shop/artykuly-spozywcze/kuchnie-swiata/produkty-kuchni-francuskiej.c-27382


2024-04-04 17:38:18,265 [DEBUG] Assigned User-Agent Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
2024-04-04 17:38:18 [scrapy_user_agents.middlewares] DEBUG: Assigned User-Agent Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.186 Safari/537.36
2024-04-04 17:38:18,267 [DEBUG] Assigned User-Agent Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36
2024-04-04 17:38:18 [scrapy_user_agents.middlewares] DEBUG: Assigned User-Agent Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36
2024-04-04 17:38:22,770 [DEBUG] Crawled (200) <GET https://zakupy.auchan.pl/api/v2/cache/products?categoryId=13612&itemsPerPage=15&page=1&cacheSegmentationCode=019_DEF&hl=pl> (referer: https://zakupy.auchan.pl/shop/artykuly-spozywcze/owoce-warzywa-ziola/ziola-i-kielki.c-136

https://zakupy.auchan.pl/api/v2/cache/products?categoryId=13612&itemsPerPage=15&page=1&cacheSegmentationCode=019_DEF&hl=pl


2024-04-04 17:38:26,946 [DEBUG] Crawled (200) <GET https://zakupy.auchan.pl/api/v2/cache/products?categoryId=27394&itemsPerPage=15&page=1&cacheSegmentationCode=019_DEF&hl=pl> (referer: https://zakupy.auchan.pl/shop/artykuly-spozywcze/kuchnie-swiata/produkty-kuchni-hiszpanskiej.c-27394)
2024-04-04 17:38:26 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://zakupy.auchan.pl/api/v2/cache/products?categoryId=27394&itemsPerPage=15&page=1&cacheSegmentationCode=019_DEF&hl=pl> (referer: https://zakupy.auchan.pl/shop/artykuly-spozywcze/kuchnie-swiata/produkty-kuchni-hiszpanskiej.c-27394)
2024-04-04 17:38:27,062 [DEBUG] {'name': 'Auchan - Go Snack kukurydza prażona o smaku chili', 'category_name': 'Przekąski różne', 'price': 1.05, 'currency': 'PLN', 'volume': '30 g', 'unit': '30 g', 'volume_info': '30 g', 'package_unit': 'kg', 'package_size': 0.03}
2024-04-04 17:38:27 [root] DEBUG: {'name': 'Auchan - Go Snack kukurydza prażona o smaku chili', 'category_name': 'Przekąski różne', 'price': 1.05

...

https://zakupy.auchan.pl/api/v2/cache/products?categoryId=27382&itemsPerPage=15&page=4&cacheSegmentationCode=019_DEF&hl=pl


2024-04-04 17:39:12,682 [DEBUG] Crawled (200) <GET https://zakupy.auchan.pl/api/v2/cache/products?categoryId=27382&itemsPerPage=15&page=3&cacheSegmentationCode=019_DEF&hl=pl> (referer: https://zakupy.auchan.pl/shop/artykuly-spozywcze/kuchnie-swiata/produkty-kuchni-francuskiej.c-27382)
2024-04-04 17:39:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://zakupy.auchan.pl/api/v2/cache/products?categoryId=27382&itemsPerPage=15&page=3&cacheSegmentationCode=019_DEF&hl=pl> (referer: https://zakupy.auchan.pl/shop/artykuly-spozywcze/kuchnie-swiata/produkty-kuchni-francuskiej.c-27382)
2024-04-04 17:39:12,791 [DEBUG] {'name': 'St Michel - Ciasteczka maślane Palets', 'category_name': 'Produkty kuchni francuskiej', 'price': 9.59, 'currency': 'PLN', 'volume': '150 g', 'unit': '150 g', 'volume_info': '150 g', 'package_unit': 'kg', 'package_size': 0.15}
2024-04-04 17:39:12 [root] DEBUG: {'name': 'St Michel - Ciasteczka maślane Palets', 'category_name': 'Produkty kuchni francuskiej', 'price': 9.5

https://zakupy.auchan.pl/api/v2/cache/products?categoryId=27382&itemsPerPage=15&page=3&cacheSegmentationCode=019_DEF&hl=pl


2024-04-04 17:39:15,601 [DEBUG] Crawled (200) <GET https://zakupy.auchan.pl/api/v2/cache/products?categoryId=27382&itemsPerPage=15&page=2&cacheSegmentationCode=019_DEF&hl=pl> (referer: https://zakupy.auchan.pl/shop/artykuly-spozywcze/kuchnie-swiata/produkty-kuchni-francuskiej.c-27382)
2024-04-04 17:39:15 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://zakupy.auchan.pl/api/v2/cache/products?categoryId=27382&itemsPerPage=15&page=2&cacheSegmentationCode=019_DEF&hl=pl> (referer: https://zakupy.auchan.pl/shop/artykuly-spozywcze/kuchnie-swiata/produkty-kuchni-francuskiej.c-27382)
2024-04-04 17:39:15,708 [DEBUG] {'name': 'Auchan - Grzanki  pieczywo pszenne trwałe z dodatkiem mąki pełnoziarnistej', 'category_name': 'Produkty kuchni francuskiej', 'price': 6.25, 'currency': 'PLN', 'volume': '225 g', 'unit': '225 g', 'volume_info': '225 g', 'package_unit': 'kg', 'package_size': 0.225}
2024-04-04 17:39:15 [root] DEBUG: {'name': 'Auchan - Grzanki  pieczywo pszenne trwałe z dodatkiem mąki peł

https://zakupy.auchan.pl/api/v2/cache/products?categoryId=27382&itemsPerPage=15&page=2&cacheSegmentationCode=019_DEF&hl=pl


Finally we can select all saved items from the DB and make a pandas dataFrame from it.

In [11]:
rows = select_all()
columns = ["product_id", "product_name", "category_name", "price", "currency", "volume", "unit", "volume_info", "package_unit", "package_size"]

df = pd.DataFrame(rows, columns=columns)
df.head()

,product_id,product_name,category_name,price,currency,volume,unit,volume_info,package_unit,package_size
0,1,Auchan - Kolendra,Kolendra,6.19,PLN,1,unit,sztuka,kg,0.25
1,2,Auchan - Bazylia,Bazylia,6.19,PLN,1,unit,sztuka,kg,0.25
2,3,Szczypiorek - Pęczek,Szczypiorek,2.48,PLN,1,unit,sztuka,szt.,1.00
3,4,Auchan - Szczypiorek,Szczypiorek,6.19,PLN,1,unit,sztuka,kg,0.25
4,5,Auchan - Szczypior sałatkowy,Szczypiorek,6.25,PLN,1,unit,sztuka,szt.,1.00
